In [1]:
# Création du dossier WebCrawler contenant l'ensemble des fichiers utiles au fonctionnement de scrapy
!scrapy startproject MangaCrawler

Error: scrapy.cfg already exists in /home/yannis/Projects/Ecole_IPSSI/2022-2023/WebScraping/Scrapy/WebScraping/MangaCrawler


In [9]:
# ***************** création du projet Manga *********

!cd MangaCrawler && scrapy genspider mangascantrad https://www.japscan.me/

Created spider 'mangascantrad' using template 'basic' in module:
  MangaCrawler.spiders.mangascantrad


In [ ]:
# ***************** Ajout au fichier items.py & Liste des données qui seront récupérés par manga *********

class MangacrawlerItem(scrapy.Item):
    # define the fields for your item here like:
    title = scrapy.Field()
    img = scrapy.Field()
    rating = scrapy.Field()
    last_chapter = scrapy.Field()
    link = scrapy.Field()
    genres = scrapy.Field()
    published_date = scrapy.Field()
    state = scrapy.Field()
    nb_comments = scrapy.Field()
    pass

In [ ]:
# ***************** Liste des commandes********************

#1. Le titre du manga
response.css('div#main div.card-body h1::text').get()

#2. Le lien de l'image du manga
response.css('div#main div.card-body img').attrib['src']

#3. L'origine du manga
response.css('div#main div.card-body p.mb-2')[1].css('span::text')[1].get()

#4. La description du manga
response.css('div#main div.card-body p.list-group-item.list-group-item-primary.text-justify::text').get()

#5. Le dernier chapitre du manga
response.css('div#chapters_list div.collapse.show div.chapters_list.text-truncate').get().split(':')[0].split('\t')[-1]

#5. Le lien de la page détail du manga
response.request.url

#8. Les genres du manga
response.css('div#main div.card-body p.mb-2')[5].get().split('</span')[1].split('\t')[7].strip()

#6. Le type du manga
response.css('div#main div.card-body p.mb-2')[4].get().split('</span')[1].split('\t')[7].strip()

#6. La date de publication du manga
response.css('div#main div.card-body p.mb-2')[3].get().split('</span')[1].split('\t')[7].strip()

#7. L'état du manga
response.css('div#main div.card-body p.mb-2')[2].get().split('</span')[1].split('\t')[6].strip()


### Récupération des liens de chaque manga



In [14]:
# La class MangascantradSpider
import scrapy
from scrapy import Request
from MangaCrawler.items import MangaGenres, MangacrawlerItem, DataBase
import sqlalchemy as db
from sqlalchemy.orm import declarative_base, relationship

class MangascantradSpider(scrapy.Spider):
    name = 'mangascantrad'
    allowed_domains = ['manga-scantrad.net']

    # Liste des urls par pages
    start_urls = [f'http://manga-scantrad.net/manga/page/{n}' for n in range(1, 31)]

    # Création de la base de données
    database = DataBase('database_manga')

    # Creation des tables avec une relation ManyToMany
    Base = declarative_base()
    association_table = db.Table(
        "mangas_assoc_genres",
        Base.metadata,
        db.Column("mangas_id", db.ForeignKey("mangas.id_")),
        db.Column("mangas_genres_id", db.ForeignKey("mangas_genres.id_")),
    )
    database.create_table('mangas_genres',
                          id_=db.Integer,
                          name=db.String,
                          )

    database.create_table('mangas',
                          id_=db.Integer,
                          title=db.String,
                          img=db.String,
                          rating=db.String,
                          last_chapter=db.String,
                          link=db.String,
                          genres=db.String,
                          published_date=db.String,
                          state=db.String,
                          nb_comments=db.String,
                          children=relationship("mangas_genres", secondary=association_table)
                          )

    def start_requests(self):
        print("iok")
        # for url in self.start_urls:
        #     yield Request(url=url, callback=self.parse_manga)

    def parse_manga(self, response):
        mangas = response.css('div.js-categories-seasonal.js-block-list.list table tr')[1:]
        for manga in mangas:
            item = MangacrawlerItem()

            # Nom manga
            try:
                item['name'] = manga.css('td')[1].css('a.hoverinfo_trigger.fw-b strong::text').get()
            except:
                item['name'] = 'None'

            # Image manga
            try:
                item['img'] = manga.css('td')[0].css('a.hoverinfo_trigger img').attrib['data-src']
            except:
                item['img'] = 'None'

            # Description manga
            try:
                item['description'] = manga.css('td')[1].css('div.pt4::text').get()
            except:
                item['description'] = 'None'

            # Ajouter dans la base de données
            self.database.add_row('manga',
                                  name=item['name'],
                                  img=item['img'],
                                  description=item['description']
                                  )
            yield item



ModuleNotFoundError: No module named 'MangaCrawler.items'

In [30]:
# Exécution de scrapy
!cd MangaCrawler/MangaCrawler/spiders && scrapy crawl mangascantrad -o mangascantrad.csv

*********Table : 'mangas_types' are created succesfully************
*********Table : 'mangas' are created succesfully************
*********Table Relationship : 'mangas_assoc_types' are created succesfully************
2022-10-20 15:39:35 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: MangaCrawler)
2022-10-20 15:39:35 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.9.2 (default, Feb 28 2021, 17:03:44) - [GCC 10.2.1 20210110], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.0-19-amd64-x86_64-with-glibc2.31
2022-10-20 15:39:35 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'MangaCrawler',
 'NEWSPIDER_MODULE': 'MangaCrawler.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['MangaCrawler.spiders']}
2022-10-20 15:39:35 [py.warnings] WARNING: /home/yannis/.local/lib/python3.9/site-packages/scrapy/utils/request.py:231: ScrapyDeprecationWarning: '2.6' is

2022-10-20 15:39:36 [scrapy.core.engine] DEBUG: Crawled (404) <GET https://www.japscan.me/manga/https://www.japscan.me/mangas/19> (referer: None)
2022-10-20 15:39:36 [scrapy.core.engine] DEBUG: Crawled (404) <GET https://www.japscan.me/manga/https://www.japscan.me/mangas/18> (referer: None)
2022-10-20 15:39:36 [scrapy.core.engine] DEBUG: Crawled (404) <GET https://www.japscan.me/manga/https://www.japscan.me/mangas/21> (referer: None)
2022-10-20 15:39:36 [scrapy.spidermiddlewares.httperror] INFO: Ignoring response <404 https://www.japscan.me/manga/https://www.japscan.me/mangas/14>: HTTP status code is not handled or not allowed
2022-10-20 15:39:36 [scrapy.core.engine] DEBUG: Crawled (404) <GET https://www.japscan.me/manga/https://www.japscan.me/mangas/23> (referer: None)
2022-10-20 15:39:36 [scrapy.core.engine] DEBUG: Crawled (404) <GET https://www.japscan.me/manga/https://www.japscan.me/mangas/22> (referer: None)
2022-10-20 15:39:36 [scrapy.core.engine] DEBUG: Crawled (404) <GET https: